In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.datasets import fetch_openml
#from sklearn.preprocessing import minmax_scale

In [2]:
#train_ds = data.astype(np.float32) / 255.0
trj = np.loadtxt('trj_place_sq20000.csv', delimiter=',')[:20000].T
train_place = np.loadtxt('spike2_place300_sq20000.csv', delimiter=',')
train_head = np.loadtxt('spike_head100_sq20000.csv', delimiter=',')
train_boundary = np.loadtxt('spike_boundary_sq20000.csv', delimiter=',').T
train_grid = np.loadtxt('spike_grid_sq10000.csv', delimiter=',').T
train_grid2 = np.loadtxt('spike_grid2_sq10000.csv', delimiter=',').T
train_speed0 = np.expand_dims(np.sqrt(trj[0]*trj[0] + trj[1]*trj[1]),0)
train_speed = np.copy(train_speed0)
print(trj.shape)
#print(train_ds.shape)
print(train_head.shape)
print(train_boundary.shape)
print(train_speed)
def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result
#train_ds = min_max(train_ds[:,::1])
trj = min_max(trj)
train_head = min_max(train_head[::2,:])
train_boundary0 = np.copy(train_boundary)
#for i in range(5):
#    train_boundary = np.append(train_boundary,train_boundary0, axis=0)
for i in range(50):
    train_speed = np.append(train_speed,train_speed0, axis=0)
print("s",train_speed.shape)
train_boundary = min_max(train_boundary)
train_place = min_max(train_place)
train_speed = min_max(train_speed[::2,:])
train_grid = min_max(train_grid)
train_grid2 = min_max(train_grid2)
train_ds = train_place[:,::1]
#train_ds = np.append(train_ds,train_ds, axis=1)
#train_ds = np.append(train_ds,trj.T, axis=1)
train_ds = np.append(train_ds,train_head.T, axis=1)
train_ds = np.append(train_ds,train_speed.T, axis=1)
#train_ds = np.append(train_ds,train_boundary.T, axis=1)
#train_ds = np.append(train_ds,train_grid2, axis=1)
print(train_ds.shape)

(2, 20000)
(100, 20000)
(8, 20000)
[[35.35533906 35.90018555 36.78017369 ... 18.70498324 17.26604406
  15.82186203]]
s (51, 20000)
(20000, 376)


In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

class Net(nn.Module):

    def __init__(self,in_dim, out_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_dim, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc21 = nn.Linear(200, 200)
        self.fc22 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, out_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc21(x))
        x = F.relu(self.fc22(x))
        x = self.fc3(x)
        return x
    
    def forward1(self, x):
        x = F.relu(self.fc1(x))
        return x
    
    def forward2(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
    
    def forward21(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc21(x))
        return x
    
    def forward22(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc21(x))
        x = F.relu(self.fc22(x))
        return x

time = 5000
X_train = train_ds[:time-1,:]
y_train = train_place[1:time,:]
x = Variable(torch.from_numpy(X_train).float(), requires_grad=True)
y = Variable(torch.from_numpy(y_train).float())
net = Net(X_train.shape[1],y_train.shape[1])
optimizer = optim.SGD(net.parameters(), lr=1.0)
criterion = nn.MSELoss()

for i in range(20000):
    optimizer.zero_grad()
    output = net(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    #with torch.no_grad():
        #j=0
        #for p in net.parameters():
            #print("i",i)
            #print(p.T.shape)
            #print(p.T)
            #j+=1
            #if j==1: #last layer weights
                #p.T[:train_place.shape[1],:].data.clamp_(0, 10.0)
            #if j==2: #last layer bias
                #p.data.clamp_(0, 0.0)
            #if j==5: #last layer weights
                #p.data.clamp_(0, 10.0)
            #if j==6: #last layer bias
                #p.data.clamp_(0, 0.0)
                #print(p)
        #for p in net.parameters():
            #print("i",i)
            #print(p.shape)
            #print(p)
    if i%500==0:
        print("i",i,"loss",loss)

# test
outputs = net(Variable(torch.from_numpy(X_train).float()))
#_, predicted = torch.max(outputs.data, 1)
#y_predicted = predicted.numpy()
#y_true = np.argmax(y_test, axis=1)
#accuracy = (int)(100 * np.sum(y_predicted == y_true) / len(y_predicted))
#print('accuracy: {0}%'.format(accuracy))


# utility function to predict for an unknown data
def predict(X):
    X = Variable(torch.from_numpy(np.array(X)).float())
    outputs = net(X)
    return np.argmax(outputs.data.numpy())

i 0 loss tensor(0.0114, grad_fn=<MseLossBackward0>)
i 500 loss tensor(0.0090, grad_fn=<MseLossBackward0>)
i 1000 loss tensor(0.0090, grad_fn=<MseLossBackward0>)


In [ ]:
outputs1 = net.forward1(Variable(torch.from_numpy(X_train).float()))
print(outputs1.shape)
#print(outputs2)

In [ ]:
v = outputs1.to('cpu').detach().numpy().copy()
o = outputs.to('cpu').detach().numpy().copy()
#print(h)
plt.figure(figsize = (12,12))
plt.imshow(X_train.T[:,:time], cmap=plt.cm.plasma,aspect='auto')
plt.xlabel('Time (s)')
plt.ylabel('Cell #')
plt.colorbar(orientation='horizontal', label='Firing Rate (Hz)')

plt.figure(figsize = (12,12))
plt.imshow(o.T, cmap=plt.cm.plasma,aspect='auto')
plt.xlabel('Time (s)')
plt.ylabel('Cell #')
plt.colorbar(orientation='horizontal', label='Firing Rate (Hz)')

In [ ]:
plt.figure(figsize = (12,12))
plt.imshow(v.T, cmap=plt.cm.plasma,aspect='auto')
plt.xlabel('Time (s)')
plt.ylabel('Cell #')
plt.colorbar(orientation='horizontal', label='Firing Rate (Hz)')

In [ ]:
for cell_num in range(v.shape[1]):
    celula = v[:,cell_num]
    fig, ax = plt.subplots()
    #mappable = ax.scatter(trj[0,:h.shape[0]], trj[1,:h.shape[0]], c=celula ,cmap='coolwarm',vmax=1,s=1)
    mappable = ax.scatter(trj[0,:v.shape[0]], trj[1,:v.shape[0]], c=celula ,cmap='coolwarm',s=1)
    fig.colorbar(mappable)
    plt.title(cell_num+1)
    plt.show()

In [ ]:
outputs2 = net.forward2(Variable(torch.from_numpy(X_train).float()))
h = outputs2.to('cpu').detach().numpy().copy()
for cell_num in range(h.shape[1]):
    celula = h[:,cell_num]
    fig, ax = plt.subplots()
    #mappable = ax.scatter(trj[0,:h.shape[0]], trj[1,:h.shape[0]], c=celula ,cmap='coolwarm',vmax=1,s=1)
    mappable = ax.scatter(trj[0,:h.shape[0]], trj[1,:h.shape[0]], c=celula ,cmap='coolwarm',s=1)
    fig.colorbar(mappable)
    plt.title(cell_num+1)
    plt.show()

In [ ]:
outputs3 = net.forward21(Variable(torch.from_numpy(X_train).float()))
h = outputs3.to('cpu').detach().numpy().copy()
for cell_num in range(h.shape[1]):
    celula = h[:,cell_num]
    fig, ax = plt.subplots()
    #mappable = ax.scatter(trj[0,:h.shape[0]], trj[1,:h.shape[0]], c=celula ,cmap='coolwarm',vmax=1,s=1)
    mappable = ax.scatter(trj[0,:h.shape[0]], trj[1,:h.shape[0]], c=celula ,cmap='coolwarm',s=1)
    fig.colorbar(mappable)
    plt.title(cell_num+1)
    plt.show()